In [98]:
import pandas as pd
import numpy as np
import re

In [5]:
from konlpy.tag import Twitter
twitter = Twitter()

C:\Users\sbg02\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [105]:
df = pd.read_csv('C:/newsdata/삼성전자.csv',  encoding = 'utf-8 sig', engine = 'python')
df.head()

,TITLE,BODY,TIME
0,삼성전자 강경훈 부사장 19일 구속심사,[theL]삼성에버랜드 노조 설립 방해·노조 와해 혐의에버랜드 노조 와해 의혹을 수...,2018.12.181953
1,檢 '노조 와해 의혹' 강경훈 삼성전자 부사장 구속영장 청구,[the L]'삼성 노조 와해 의혹'을 수사 중인 검찰이 강경훈 삼성전자 부사장에 ...,2018.12.181943
2,"""사회가 건강해야 기업도 발전"" 20년 넘은 삼성 나눔의 비결",[특집]삼성전자는 1995년 국내 기업 최초로 사회봉사단을 창단 사회문제 해결을 위...,2018.12.181830
3,"4차 산업 혁신기업 인탑스, 스마트팩토리 중소벤처기업부 장관상 수상",인탑스(주)(대표 김근하 정사진)가 스마트팩토리 도입을 추진해 경북창조경제혁신센터가...,&2018.12.181745
4,'에버랜드 노조와해' 삼성 노사총괄 부사장 내일 영장심사,염호석씨 시신탈취 과정서 뇌물수수 경찰관도 함께(서울=뉴스1) 나연준 기자 = 삼성...,12018.12.181710


### TIME클렌징1 - TIME칼럼 EDA
1. for문 => 너무 복잡 & 시간이 필요<br>
<코드>  ???
2. RE => str으로 변환 뒤 '12018.' => '2018.'  으로 바꿔야 하는데 결과값이 Series가 아니라 \t처리된 str으로 나온다.<br>
<코드> pre3 = re.sub('[1\d]{4}8\.', '2018.', str(pre1))

### (아래에서 끌어온 결과)
#### 데이터 3등분 
<코드> time['year'].unique()

=> 결과<br>
array(['2018', '12018', '-2018', '', '--2018', '3002018', '22018',
       '+2018', '122018', '2017', '12017', '-2017', '22017', '112017',
       '122017']<br><br>
=> 뒤 숫자 4개만 남기면 정리될 듯?

### 연도 클렌징 - 특수문자 &클렌징    (뒤에 더 좋은 방법이 있어서 불필요할듯...)

In [141]:
pre1 = pd.Series(df['TIME']).str.replace('&', '')
pre1.head()

0     2018.12.181953
1     2018.12.181943
2     2018.12.181830
3     2018.12.181745
4    12018.12.181710
Name: TIME, dtype: object

### 연도 클렌징 - '.' 기준으로 칼럼 3등분 나누어 접근

In [171]:
time = pd.DataFrame(pre1.str.split('.',2).tolist(), columns = ['year','month','time'])
time.head()

,year,month,time
0,2018,12,181953
1,2018,12,181943
2,2018,12,181830
3,2018,12,181745
4,12018,12,181710


In [147]:
'''
https://www.geeksforgeeks.org/python-pandas-series-str-slice/
year_clean = time.year[-4:]

for i in range(0,len(df)):
    df.iloc[i].col = df.iloc[i].col[:9]
    
df.col = df.col.str[:9]
    
    
    '''

8276    2017
8277    2017
8278    2017
8279    2017
Name: year, dtype: object

### 연도 클렌징(실패) - 슬라이싱한 부분을 삭제하는 코드....ㅠ

In [156]:
'''
# start stop and step variables 
start, stop, step = 0, -4, 1
  
# converting to string data type 
data["Salary"]= data["Salary"].astype(str) 
  
# slicing till 4rd last element 
time['year'] = time['year'].str.slice(start, stop, step) 
  
# display 
time.head()
'''

,year,month,time
0,,12,181953
1,,12,181943
2,,12,181830
3,,12,181745
4,1,12,181710


### 연도 클렌징(성공) - 람다 사용해 단순 백슬라이싱

In [172]:
time['year'] = time['year'].apply(lambda x: x[-4:])
time['year'].unique()

array(['2018', '', '2017'], dtype=object)

### 시간 클렌징 - 초 삭제

In [173]:

# start stop and step variables 
start, stop, step = 0, -2, 1
  
# slicing till 2nd last element 
time['time'] = time['time'].str.slice(start, stop, step) 
  
# display 
time.head()

,year,month,time
0,2018,12,1819
1,2018,12,1819
2,2018,12,1818
3,2018,12,1817
4,2018,12,1817


### 마무리 - 원본에 붙이기

In [177]:
# 칼럼 삭제
del df['TIME']

KeyError: 'TIME'

In [180]:
# 칼럼 합치기
df_timeok = pd.concat([df, time], axis=1, sort=False)
df_timeok

,TITLE,BODY,year,month,time
0,삼성전자 강경훈 부사장 19일 구속심사,[theL]삼성에버랜드 노조 설립 방해·노조 와해 혐의에버랜드 노조 와해 의혹을 수...,2018,12,1819
1,檢 '노조 와해 의혹' 강경훈 삼성전자 부사장 구속영장 청구,[the L]'삼성 노조 와해 의혹'을 수사 중인 검찰이 강경훈 삼성전자 부사장에 ...,2018,12,1819
2,"""사회가 건강해야 기업도 발전"" 20년 넘은 삼성 나눔의 비결",[특집]삼성전자는 1995년 국내 기업 최초로 사회봉사단을 창단 사회문제 해결을 위...,2018,12,1818
3,"4차 산업 혁신기업 인탑스, 스마트팩토리 중소벤처기업부 장관상 수상",인탑스(주)(대표 김근하 정사진)가 스마트팩토리 도입을 추진해 경북창조경제혁신센터가...,2018,12,1817
4,'에버랜드 노조와해' 삼성 노사총괄 부사장 내일 영장심사,염호석씨 시신탈취 과정서 뇌물수수 경찰관도 함께(서울=뉴스1) 나연준 기자 = 삼성...,2018,12,1817
5,"文 ""수소차에 획기적 뒷받침…비장한 각오로 제조업 부흥""","[the300]""수소차 국내 수요 늘려야…전기차 지원예산 여유있게 확보""문재인 대통...",2018,12,1816
6,"코스피, 외인·개인 매도에 2060선으로 밀려",[시황종합]코스피 전일 대비 8.98p(0.43%) 내린 2062.11 마감코스피가...,2018,12,1816
7,"삼성전자, 美 연말 초대형 TV 판매 웃었다…中·日 추격 따돌려",베스트바이 75형 이상 초대형 TV '베스트 셀링'…75형 UHD 82형 QLED ...,2018,12,1814
8,"[부자들은 경제신문을 읽는다] 김성훈 팀장 ""테마주, 고점 매수No… 손절가 확인 ...",부~~자 되는 이야기만 '쏙쏙' 뽑아 알려드립니다.오늘 머니투데이의 가장 핫한 증시...,2018,12,1814
9,"[돈 잘버는 증권맨에게 듣는다] 이재광 이사 ""아난티 이상 급등… 과감히 매수?""",돈 잘 버는 증권맨이 증시 분석과 투자 전략을 알려드립니다.개장 후 주요 이슈를 체...,2018,12,1814


# 시간정제 끝!!

In [12]:
data = twitter.nouns(u'그렇게 기대했던 취업이었지만, 막상 나에게 펼쳐진건 본 게임이었다.')
data

['취업', '막상', '나', '게임']

In [71]:
df1.apply(lambda x : twitter.nouns(x))

AttributeError: 'str' object has no attribute 'apply'

In [66]:
df['TITLE'].apply(lambda x : twitter.nouns(x))

0                             [삼성, 전자, 강경, 훈, 부사, 구속, 심사]
1                   [노조, 의혹, 강경, 훈, 삼성, 전자, 부사, 구속영장, 청구]
2                                [사회, 기업, 발전, 삼성, 나눔, 비결]
3       [차, 산업, 혁신, 기업, 인탑스, 스마트, 팩토리, 중소, 벤처기업, 부, 장관...
4               [에버랜드, 노조, 해, 삼성, 노사, 총괄, 부사, 내일, 영장, 심사]
5                             [수소차, 획기, 뒷받침, 각오, 제조업, 부흥]
6                                    [코스피, 외인, 개인, 매도, 선]
7                               [삼성, 전자, 연말, 초대형, 판매, 추격]
8         [부자, 신문, 김성훈, 팀, 테마주, 점, 매수, 손절, 확인, 후, 매수, 전략]
9       [돈, 잘버, 증권, 맨, 이재, 광, 이사, 아난, 티, 이상, 급등, 과, 감히...
10                         [제조업, 혁신, 노동자, 기업, 양보, 상생, 모델]
11                                [이스탄불, 공항, 장식, 삼성, 사이니]
12                                 [방, 안, 옷가게, 삼성, 패션, 집]
13                         [특징, 주, 삼성, 전자, 사흘, 주, 신저, 경신]
14                                 [뉴욕증시, 하락, 코스피, 선, 약세]
15                                [리포트, 화웨이, 왜, 공공, 적, 이]
16                [부고, 원빈, 머니투데이, 국제, 부, 디자인, 기자, 부친상, 외]
17            

In [ ]:
df['name'] = df['name'].apply(lambda e: e.split()[0])

>>> from timeit import Timer
>>> t = {}
>>> t['methodcaller'] = Timer("map(methodcaller('split', ' '), a)", "from operator import methodcaller; a=['2011-12-22 46:31:11','2011-12-20 20:19:17', '2011-12-20 01:09:21']")
>>> t['lambda'] = Timer("map(lambda s: s.split(), a)", "a = ['2011-12-22 46:31:11','2011-12-20 20:19:17', '2011-12-20 01:09:21']")
>>> t['listcomp'] = Timer("[s.split() for s in a]", "a = ['2011-12-22 46:31:11','2011-12-20 20:19:17', '2011-12-20 01:09:21']")
>>> for name, timer in t.items():
...     print '%s: %.2f usec/pass' % (name, 1000000 * timer.timeit(number=100000)/100000)
... 
listcomp: 2.08 usec/pass
methodcaller: 2.87 usec/pass
lambda: 3.10 usec/pass